In [4]:
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
import skimage.transform
import requests
from io import BytesIO
import os
%matplotlib inline

In [6]:
dir_name = '/Users/Htoo Min/Documents/GitHub/PizzaPie/Test'
for photos in os.listdir(dir_name):
    print(photos)

DSCN5066.JPG
entree pizza.jpg
exchange200901101539428ebad3ba0598a06cf38f9167bebbdd3a388.jpg
frannys-pizza.jpg
frozen-pizza-04-Newmans.jpg
fugazetta06.jpg
fugazzeta 009.jpg
f_4012159_1.JPG
gallery_20347_716_262199.jpg
gallery_20347_716_387672.jpg
gallery_21337_4990_43981.jpg
gallery_21337_4990_97823.jpg
give-pizza-pizza - Shortcut.lnk
give-pizza-pizza.jpg
greenbean mushroom onion bluecheesepizza.jpg
grilledpizzawithhotsausage_e.jpg
HomeRunInnSausagePizza_web.jpg
I61_pizza1.jpg
il fornello sausage pizza.jpg
image1919573g.jpg
Imagen 083.jpg
Imagen 602 copia.jpg
Imagen 606 copia.jpg
img_0054-225x300.jpg
img_0450.jpg
IMG_0777.JPG
img_0912.jpg
img_0922.jpg
img_1420.jpg
IMG_2336-225x300.jpg
IMG_2548-225x300.jpg
IMG_2663-768x1024.jpg
IMG_5012.JPG
IMG_6083.JPG
istock_sicilian_pizza_small - Shortcut.lnk
istock_sicilian_pizza_small.jpg
Italian Sausage and olive Pizza October 19th, 2009 1.jpg
JGALATIOTO_IMG_0085.jpg
JO1B26_24609_s4x3_lg.jpg
Joes pizza behind glass2.JPG
Jonica Pizza.jpg
kashi pizza

In [7]:
import os
dir_exists = os.path.isdir(dir_name)
if not dir_exists:
    os.mkdir(dir_name)
    print("Making directory %s" % dir_name)
else:
    print("Will store images in directory %s" % dir_name)

Will store images in directory /Users/Htoo Min/Documents/GitHub/PizzaPie/Test


In [8]:
# Display the image
def disp_image(im):
    if (len(im.shape) == 2):
        # Gray scale image
        plt.imshow(im, cmap='gray')    
    else:
        # Color image.  
        im1 = (im-np.min(im))/(np.max(im)-np.min(im))*255
        im1 = im1.astype(np.uint8)
        plt.imshow(im1)    
        
    # Remove axis ticks
    plt.xticks([])
    plt.yticks([])

In [20]:
import warnings   
nimage = 210
i = 0
nrow = 224
ncol = 224
#Converts the names of the images
for photo in os.listdir(dir_name):
    print("path", dir_name, "filename", photo, dir_name+"/"+photo)  
    
    # Read image from file
    im = skimage.io.imread(dir_name+"/"+photo)
        
    # Resize images
    im1 = skimage.transform.resize(im,(nrow,ncol),mode='constant')
                
    # Convert to uint8, suppress the warning about the precision loss
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        im2 = skimage.img_as_ubyte(im1)
    
    # Save the image
    local_name = '{0:s}/{1:s}_{2:04d}.jpg'.format(dir_name,"pizza", i)  
    print("lcl", local_name)
    skimage.io.imsave(local_name, im2)      
    print(local_name)
    i = i + 1        
    if (i >= nimage):        
        break

path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename DSCN5066.JPG /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/DSCN5066.JPG
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0000.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0000.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename entree pizza.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/entree pizza.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0001.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0001.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename exchange200901101539428ebad3ba0598a06cf38f9167bebbdd3a388.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/exchange200901101539428ebad3ba0598a06cf38f9167bebbdd3a388.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0002.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0002.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename frannys-pizza.jpg /Users/Htoo Min/Documents/Git

lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0030.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0030.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename IMG_5012.JPG /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/IMG_5012.JPG
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0031.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0031.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename IMG_6083.JPG /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/IMG_6083.JPG
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0032.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0032.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename istock_sicilian_pizza_small.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/istock_sicilian_pizza_small.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0033.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0033.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/T

lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0062.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0062.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename parmigiana.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/parmigiana.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0063.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0063.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename Patsys-SausagePizza.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/Patsys-SausagePizza.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0064.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0064.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename PC040421.JPG /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/PC040421.JPG
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0065.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0065.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename

/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0093.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza (4).jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza (4).jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0094.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0094.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza (40).jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza (40).jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0095.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0095.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza (42).jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza (42).jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0096.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0096.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza (43).jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza (43).jpg
lcl 

lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0124.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0124.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza-carbonara.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza-carbonara.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0125.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0125.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza-challenge-pie.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza-challenge-pie.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0126.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0126.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza-challenge-slice.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza-challenge-slice.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0127.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0127.jpg
path /Users/Htoo Min/Doc

/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0156.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename PizzaGourmet2.gif /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/PizzaGourmet2.gif
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0157.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0157.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizzamargarita.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizzamargarita.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0158.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0158.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename PizzaMargherita.JPG /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/PizzaMargherita.JPG
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0159.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0159.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizzamenu_8.gif /Users/Htoo Min/Documents/GitHub/PizzaP

/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0189.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza_0020.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0020.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0190.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0190.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza_0021.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0021.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0191.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0191.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza_0022.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0022.jpg
lcl /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0192.jpg
/Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0192.jpg
path /Users/Htoo Min/Documents/GitHub/PizzaPie/Test filename pizza_0023.jpg /Users/Htoo Min/Documents/GitHub/PizzaPie/Test/pizza_0023.jpg
lc